In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
from scipy.stats import gamma, uniform


In [226]:
disasters = genfromtxt('disasters.csv', delimiter = ',')

def init_mcmc(N, num_breakpoints, psi, rho):

    
    #Prior for t is a logic statement
    # If t is sorted the prior is proportional to the sumproduct of differences
    # returns zero if not sorted
    prior_t_log = lambda t: np.sum(np.log(np.diff(t))) if np.all(t[:-1] <= t[1:]) else 0
    
    #The probability of tau given parameters
    f_tau_log = lambda t, lambdas, nt: -np.sum(np.diff(t)*lambdas) + np.sum(nt*np.log(lambdas))
    
    #Initialize arrays
    breakpoints = np.arange(1658,1981, (1980-1658)/(num_breakpoints+1))
    tau = disasters
    
    burn_in = 0
    M = N + burn_in

    lambdas = np.zeros((M, num_breakpoints + 1))
    thetas = np.zeros((M, num_breakpoints + 1))
    
    prob_breakpoints = np.zeros((M, num_breakpoints))
    

    for index in range(M):
        print(f'Iter: {index}')
        thetas[index] = gamma(2, psi).rvs(num_breakpoints + 1)
        for section in range(num_breakpoints + 1):
            lambdas[index, section] = gamma(2, thetas[index, section]).rvs(1)

        print(f' Theta: {thetas[index]}')
        print(f' Lambda: {lambdas[index]}')
        #Random walk proposal
        for t in np.arange(num_breakpoints) + 1:
            print(f'Breakpoint {t}')
            
            ti = breakpoints[t]
            R = rho*(breakpoints[t+1] - breakpoints[t-1])
            epsilon = np.random.uniform(-R, R)
            ti_proposal = ti+epsilon
            breakpoints_propose = np.copy(breakpoints)
            breakpoints_propose[t] = ti_proposal
            
            print(f' Current breakpoints: {breakpoints}')
            print(f' Proposed breakpoints: {breakpoints_propose}')
            
            if(np.all(breakpoints_propose[:-1] <= breakpoints_propose[1:])):
            
                #Num incidents per interval
                nt_current = np.histogram(tau, bins = breakpoints)[0].astype('int32')
                print(f'nt current: {nt_current}')
                nt_propose = np.histogram(tau, bins = breakpoints_propose)[0].astype('int32')
                print(f'nt propose: {nt_propose}')
                
                
                #Probability of accept
                p_propose_log = prior_t_log(breakpoints_propose) + f_tau_log(breakpoints_propose, lambdas, nt_propose)
                print(f'p propose: {np.exp(p_propose_log)}')
                p_current_log = prior_t_log(breakpoints) + f_tau_log(breakpoints, lambdas, nt_current)
                print(f'p current: {np.exp(p_current_log)}')
                p_accept = np.minimum(1, np.exp(p_propose_log-p_current_log))

                accept = uniform(0,1).rvs() < p_accept
                print(f' P(accept) = {p_accept}')

                if accept:
                    breakpoints = breakpoints_propose

    return breakpoints
    

In [238]:
f_tau_log = lambda t, lambdas, nt: -np.sum(np.diff(t)*lambdas) + np.sum(nt*np.log(lambdas))
lambdas = [4, 4.8]
nt = [79, 672]
t = [1658, 1819, 1980]
f_tau_log(t, lambda)

<function __main__.<lambda>(t, lambdas, nt)>

In [233]:
init_mcmc(2, 1, 0.01, 0.5)

Iter: 0
 Theta: [1.64216384 1.77830976]
 Lambda: [4.01606662 4.82442823]
Breakpoint 1
 Current breakpoints: [1658. 1819. 1980.]
 Proposed breakpoints: [1658.         1971.80774317 1980.        ]
nt current: [ 79 672]
nt propose: [745   6]
p propose: 0.0
p current: 0.0
 P(accept) = nan
Iter: 1
 Theta: [1.70504395 3.14238982]
 Lambda: [8.50165016 4.68616046]
Breakpoint 1
 Current breakpoints: [1658. 1819. 1980.]
 Proposed breakpoints: [1658.         1872.49319966 1980.        ]
nt current: [ 79 672]
nt propose: [416 335]
p propose: 0.0
p current: 0.0
 P(accept) = 2.6610735028861527e-10


<ipython-input-226-b3960b9aca51>:12: RuntimeWarning: divide by zero encountered in log
  f_tau_log = lambda t, lambdas, nt: -np.sum(np.diff(t)*lambdas) + np.sum(nt*np.log(lambdas))
<ipython-input-226-b3960b9aca51>:63: RuntimeWarning: invalid value encountered in double_scalars
  p_accept = np.minimum(1, np.exp(p_propose_log-p_current_log))


array([1658., 1819., 1980.])

In [113]:
num_breakpoints = 1
breakpoints = np.arange(1658,1981, (1980-1658)/(num_breakpoints+1))
nt = np.histogram(disasters, bins = breakpoints)[0].astype('int32')
lambdas = np.array([1, 2])

array([ 79, 672])

In [163]:
np.arange(1658,1981, (1980-1658)/(num_breakpoints+1))

array([1658., 1819., 1980.])

In [174]:
np.arange(1)

array([0])